Examples from the book series visual quantum mechanics. by Bernd Thaller <br>
<br> 
Modified    By    Reason <br> 
--------    ---   ------ <br>
01-Aug-23   CBL   Original <br> 
10-Aug-23   CBL   Modified to use A complex to hsv converter. 
<hr> 
References: <br> 
https://vqm.uni-graz.at/ <br> 
https://matplotlib.org/stable/gallery/mplot3d/surface3d.html <br>
https://stackoverflow.com/questions/24897681/python-2d-color-map-with-imshow <br>
https://matplotlib.org/3.1.0/gallery/images_contours_and_fields/pcolormesh_levels.html#sphx-glr-gallery-images-contours-and-fields-pcolormesh-levels-py <br>
https://www.tutorialspoint.com/how-to-plot-a-smooth-2d-color-plot-for-z-f-x-y-in-matplotlib <br>
https://stackoverflow.com/questions/37520251/plotting-hsv-values-with-imshow <br>
https://docs.python.org/3/library/colorsys.html<br>
https://matplotlib.org/stable/tutorials/colors/colormapnorms.html <br>
https://en.wikipedia.org/wiki/HSL_and_HSV#:~:text=The%20difference%20between%20HSL%20and,red%2C%20just%20brighter%20and%20more <br> 
This process has a name: "domain coloring", https://en.wikipedia.org/wiki/Domain_coloring <br> 
<hr>

In [5]:
import numpy as np
import matplotlib.pyplot as plt
from   matplotlib.colors import hsv_to_rgb
import colorsys as cs
import matplotlib.colors as mcolors

What I did not appreciate is that Visual Quantum Mechanics uses HLS Hue, Lightness, Saturation as compared
with HSV Hue, Saturation Value. In both instances Hue is described by an angle around a circle. Saturation is {0:100}% in both cases and is the magnitude of the vector representing the point on the 
circle. Value goes from {0 Black to 100% White}. HLS the Lightness is the {0Black to 1White} Which now that I write it seems the same. <br> 
https://support.ptc.com/help/mathcad/r9.0/en/index.html#page/PTC_Mathcad_Help/hls_and_hsv_color_representations.html
<hr>
In the book, poles = nan = white, zeros = 0 = black, lets implement this. 

Trying to create an HSV plot where: <br>
$z = x + jy $ <br>
A complex number z can be represented by its "absolute value" (or "modulus") $|z|$ and by its "argument" (or "phase") $arg(z)$. The absolute value is the distance of z from the origin in the complex plane, arg(z) is the angle between the line from (0,0) to (x,y) and the positive x-axis. The polar representation of z reads: <br>
$Z = |z|e^{i*arg(z)}$<br>

The color map uses the HLS color system ("hue-lightness-saturation"). The hue represents the argument arg(z) of the complex number z. The absolute value abs(z) is given by the lightness of the color. All colors of the color map have the maximal saturation (with respect to the given lightness).

In [6]:
def my_hls_to_rgb(h,l,s):
    # Assume they are all the same length. 
    # allocate return array
    rgb = np.zeros([len(h),3])
    for i in range(len(h)):
        rgb[i] = cs.hls_to_rgb(h[i],l[i],s[i])
    return rgb

In [13]:
def complex_to_rgb3(zz):
    """
    input a complex number and return an rgb value
    @param zz input complex value, assume a square matrix
    This is going to be fed into hsv_to_rgb which claims that each input is bound between 0:1
    This uses an HLS type of format which is in the book. HLS really only differs in that for HSV V{0:1}
    HLS -127:128 or -.5 to .5
    Normalizaton isn't quite right
    """
    norm   = mcolors.Normalize()
    norm2  = mcolors.PowerNorm(gamma=0.7)
    #norm2  = mcolors.LogNorm(gamma=0.9)
    #norm   = mcolors.CenteredNorm()
    #norm   = mcolors.SymLogNorm(linthresh=0.03, linscale=0.03, vmin=-1.0, vmax=1.0, base=10)
    if (zz.ndim>1):
        # this works better than angle and gives a better reproduction
        hue        = norm(np.arctan2(np.imag(zz),-np.real(zz)))
        saturation = np.absolute(zz)
        lightness  = np.ones(len(saturation))
        rgb = my_hls_to_rgb(hue, saturation, lightness)
        
    return rgb

In [14]:
def G(x, y):
    s = x + 1j*y
    return s

In [15]:
n = 100
x = np.linspace(-3, 3, n)
y = np.linspace(-3, 3, n)

xx, yy = np.meshgrid(x, y)
zz     = G(xx, yy)
i,j = np.where(zz.real==0)

rgb     = complex_to_rgb3(zz)
ij = np.where(rgb<0)
print(np.abs(zz[:,49]))
# plotting
plt.imshow(rgb, extent=[-3, 3, -3, 3])
plt.grid()
plt.show()

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

This matches the book.<br>
<img src="photo/colormap.jpg"> <br>
I don't quite get the black in the center. also, the intensity is greater near zero. 

In [105]:
# fool around with hsv to rgb
norm   = mcolors.Normalize()
x = 2
y = 2
angle = np.arctan2(y,-x)
hue   = norm(angle)
r = np.sqrt(np.power(x,2)+np.power(y,2))
saturation = r/100 #np.absolute(r)
value = 0
print("hue = " , hue, " saturation = ", saturation, " value = ", value)
rgb = cs.hsv_to_rgb(hue,saturation,value)
print("RGB : ", rgb)
print("RGB from HLS ", cs.hls_to_rgb(hue, value, saturation))

hue =  0.0  saturation =  0.0282842712474619  value =  0
RGB :  (0, 0.0, 0.0)
RGB from HLS  (0.0, 0.0, 0.0)
